In [2]:
import re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize

# Read the text file
with open('input_text.txt', 'r') as file:
    text = file.read()

# Clean the text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.strip()

cleaned_text = clean_text(text)

# Tokenize into sentences
sentences = sent_tokenize(cleaned_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:

# Split the text into chunks

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_text(cleaned_text)




In [4]:

# Extract entities from the text
from transformers import pipeline

ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def extract_entities(chunk):
    entities = ner_pipeline(chunk)
    return [entity['word'] for entity in entities if entity['entity'] in ['B-PER', 'B-ORG', 'B-LOC']]

concepts = [extract_entities(chunk) for chunk in chunks]


/root/P-KEvoGraphRAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [6]:



# Extract embeddings for the chunks
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)  # Not used directly here, but useful for future enhancements


relationships = []
for chunk_concepts in concepts:
    for i in range(len(chunk_concepts)):
        for j in range(i + 1, len(chunk_concepts)):
            relationships.append((chunk_concepts[i], chunk_concepts[j]))



print(concepts)

[[]]


In [7]:
# Extract concepts
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def extract_entities(chunk):
    entities = ner_pipeline(chunk)
    extracted = [entity['word'] for entity in entities if entity['entity'] in ['B-PER', 'B-ORG', 'B-LOC']]
    print(f"Chunk: {chunk[:50]}... | Entities: {extracted}")  # Debugging
    return extracted

concepts = [extract_entities(chunk) for chunk in chunks]
print(f"Concepts: {concepts}")  # Debugging

# Extract relationships
relationships = []
for chunk_concepts in concepts:
    print(f"Chunk concepts: {chunk_concepts}")  # Debugging
    for i in range(len(chunk_concepts)):
        for j in range(i + 1, len(chunk_concepts)):
            relationships.append((chunk_concepts[i], chunk_concepts[j]))

print(f"Relationships: {relationships}")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Chunk: Songlin Zhao Curriculum Vitae  February 2025 A BIO... | Entities: []
Concepts: [[]]
Chunk concepts: []
Relationships: []


In [8]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
text = "Songlin Zhao Curriculum Vitae February 2025 A BIOGRAPHY"

entities = ner_pipeline(text)
print("Raw entities:", entities)
extracted = [entity['word'] for entity in entities if entity['entity'].startswith('B-')]
print("Extracted entities:", extracted)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Raw entities: [{'entity': 'I-ORG', 'score': np.float32(0.8847604), 'index': 1, 'word': 'Song', 'start': 0, 'end': 4}, {'entity': 'I-ORG', 'score': np.float32(0.78380275), 'index': 2, 'word': '##lin', 'start': 4, 'end': 7}, {'entity': 'I-ORG', 'score': np.float32(0.8048907), 'index': 3, 'word': 'Zhao', 'start': 8, 'end': 12}, {'entity': 'I-ORG', 'score': np.float32(0.6209739), 'index': 14, 'word': 'B', 'start': 46, 'end': 47}, {'entity': 'I-MISC', 'score': np.float32(0.36976373), 'index': 15, 'word': '##IO', 'start': 47, 'end': 49}]
Extracted entities: []


In [15]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from langchain.text_splitter import CharacterTextSplitter
from transformers import pipeline
from neo4j import GraphDatabase

# Read and clean text
with open('input_text.txt', 'r') as file:
    text = file.read()

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

cleaned_text = clean_text(text)
sentences = sent_tokenize(cleaned_text)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chunks = text_splitter.split_text(cleaned_text)
print(f"Number of chunks: {len(chunks)}")

# Extract concepts with a better NER model
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")
def extract_entities(chunk):
    entities = ner_pipeline(chunk)
    extracted = []
    current_entity = []
    for entity in entities:
        if entity['entity'].startswith('B-'):
            if current_entity:
                extracted.append(" ".join(current_entity))
            current_entity = [entity['word']]
        elif entity['entity'].startswith('I-'):
            current_entity.append(entity['word'])
    if current_entity:
        extracted.append(" ".join(current_entity))
    print(f"Chunk: {chunk[:50]}... | Entities: {extracted}")
    return extracted

concepts = [extract_entities(chunk) for chunk in chunks]
print(f"Concepts: {concepts}")

# Extract relationships
relationships = []
for chunk_concepts in concepts:
    print(f"Chunk concepts: {chunk_concepts}")
    for i in range(len(chunk_concepts)):
        for j in range(i + 1, len(chunk_concepts)):
            relationships.append((chunk_concepts[i], chunk_concepts[j]))

print(f"Relationships: {relationships}")

# Store in Neo4j
driver = GraphDatabase.driver("bolt://45.55.34.153:7687", auth=("neo4j", "password"))

def add_concept(tx, name):
    tx.run("MERGE (c:Concept {name: $name})", name=name)

def add_relationship(tx, concept1, concept2):
    tx.run("MATCH (c1:Concept {name: $concept1}), (c2:Concept {name: $concept2}) "
           "MERGE (c1)-[:RELATED_TO]->(c2)", concept1=concept1, concept2=concept2)

with driver.session() as session:
    for concept in set(sum(concepts, [])):
        session.write_transaction(add_concept, concept)
    for rel in relationships:
        session.write_transaction(add_relationship, rel[0], rel[1])

driver.close()
print("Knowledge graph stored in Neo4j!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Number of chunks: 1
Chunk: **Songlin Zhao** **Curriculum Vitae - February 202... | Entities: ['Song', '##lin Zhao', 'C Vita ##AP Drive Lehigh University Bethlehem', 'PA', 'Computer Science', 'Lehigh University', 'Bethlehem', 'PA', 'Li He', 'Research Inter', 'Lehigh University', 'Bethlehem', 'PA', 'Li', '##o Sun', 'B . S', 'Computer Science', 'Tong ##ji University', 'Shanghai', 'China', 'Da', '##wei Cheng', 'Xi Lu', 'Song', '##lin Zhao', 'Yun Z ##ong', 'Yong Chen', 'G', '##ui Shu ##ang Ying', 'Li', '##g He', 'Re ##l Disease', 'AAA', 'AI for Health', 'B', '##ur ##ling ##ame', 'California', 'USA', 'Song ##lin Zhao', 'Ron', '##g Zhou', 'Yu Zhang', 'Yong Chen', 'B', 'Computing', 'AI Springer', 'Chen', 'Li']
Concepts: [['Song', '##lin Zhao', 'C Vita ##AP Drive Lehigh University Bethlehem', 'PA', 'Computer Science', 'Lehigh University', 'Bethlehem', 'PA', 'Li He', 'Research Inter', 'Lehigh University', 'Bethlehem', 'PA', 'Li', '##o Sun', 'B . S', 'Computer Science', 'Tong ##ji University', 'S

/tmp/ipykernel_147461/2578048935.py:68: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_concept, concept)
/tmp/ipykernel_147461/2578048935.py:70: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_relationship, rel[0], rel[1])


Knowledge graph stored in Neo4j!
